In [84]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00


In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import chi2_contingency
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

from sklearn.naive_bayes import GaussianNB


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

df = pd.concat([train, test], axis=0)

In [5]:
train.head()

,obs,job_title,job_posted_date,salary_category,job_state,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,job_desc_001,job_desc_002,job_desc_003,job_desc_004,job_desc_005,job_desc_006,job_desc_007,job_desc_008,job_desc_009,job_desc_010,job_desc_011,job_desc_012,job_desc_013,job_desc_014,job_desc_015,job_desc_016,job_desc_017,job_desc_018,job_desc_019,job_desc_020,job_desc_021,job_desc_022,job_desc_023,job_desc_024,job_desc_025,job_desc_026,job_desc_027,job_desc_028,job_desc_029,job_desc_030,job_desc_031,job_desc_032,job_desc_033,job_desc_034,job_desc_035,job_desc_036,job_desc_037,job_desc_038,job_desc_039,job_desc_040,job_desc_041,job_desc_042,job_desc_043,job_desc_044,job_desc_045,job_desc_046,job_desc_047,job_desc_048,job_desc_049,job_desc_050,job_desc_051,job_desc_052,job_desc_053,job_desc_054,job_desc_055,job_desc_056,job_desc_057,job_desc_058,job_desc_059,job_desc_060,job_desc_061,job_desc_062,job_desc_063,job_desc_064,job_desc_065,job_desc_066,job_desc_067,job_desc_068,job_desc_069,job_desc_070,job_desc_071,job_desc_072,job_desc_073,job_desc_074,job_desc_075,job_desc_076,job_desc_077,job_desc_078,job_desc_079,job_desc_080,job_desc_081,job_desc_082,job_desc_083,job_desc_084,job_desc_085,job_desc_086,job_desc_087,job_desc_088,job_desc_089,job_desc_090,job_desc_091,job_desc_092,job_desc_093,job_desc_094,job_desc_095,job_desc_096,job_desc_097,job_desc_098,job_desc_099,job_desc_100,job_desc_101,job_desc_102,job_desc_103,job_desc_104,job_desc_105,job_desc_106,job_desc_107,job_desc_108,job_desc_109,job_desc_110,job_desc_111,job_desc_112,job_desc_113,job_desc_114,job_desc_115,job_desc_116,job_desc_117,job_desc_118,job_desc_119,job_desc_120,job_desc_121,job_desc_122,job_desc_123,job_desc_124,job_desc_125,job_desc_126,job_desc_127,job_desc_128,job_desc_129,job_desc_130,job_desc_131,job_desc_132,job_desc_133,job_desc_134,job_desc_135,job_desc_136,job_desc_137,job_desc_138,job_desc_139,job_desc_140,job_desc_141,job_desc_142,job_desc_143,job_desc_144,job_desc_145,job_desc_146,job_desc_147,job_desc_148,job_desc_149,job_desc_150,job_desc_151,job_desc_152,job_desc_153,job_desc_154,job_desc_155,job_desc_156,job_desc_157,job_desc_158,job_desc_159,job_desc_160,job_desc_161,job_desc_162,job_desc_163,job_desc_164,job_desc_165,job_desc_166,job_desc_167,job_desc_168,job_desc_169,job_desc_170,job_desc_171,job_desc_172,job_desc_173,job_desc_174,job_desc_175,job_desc_176,job_desc_177,job_desc_178,job_desc_179,job_desc_180,job_desc_181,job_desc_182,job_desc_183,job_desc_184,job_desc_185,job_desc_186,job_desc_187,job_desc_188,job_desc_189,job_desc_190,job_desc_191,job_desc_192,job_desc_193,job_desc_194,job_desc_195,job_desc_196,job_desc_197,job_desc_198,job_desc_199,job_desc_200,job_desc_201,job_desc_202,job_desc_203,job_desc_204,job_desc_205,job_desc_206,job_desc_207,job_desc_208,job_desc_209,job_desc_210,job_desc_211,job_desc_212,job_desc_213,job_desc_214,job_desc_215,job_desc_216,job_desc_217,job_desc_218,job_desc_219,job_desc_220,job_desc_221,job_desc_222,job_desc_223,job_desc_224,job_desc_225,job_desc_226,job_desc_227,job_desc_228,job_desc_229,job_desc_230,job_desc_231,job_desc_232,job_desc_233,job_desc_234,job_desc_235,job_desc_236,job_desc_237,job_desc_238,job_desc_239,job_desc_240,job_desc_241,job_desc_242,job_desc_243,job_desc_244,job_desc_245,job_desc_246,job_desc_247,job_desc_248,job_desc_249,job_desc_250,job_desc_251,job_desc_252,job_desc_253,job_desc_254,job_desc_255,job_desc_256,job_desc_257,job_desc_258,job_desc_259,job_desc_260,job_desc_261,job_desc_262,job_desc_263,job_desc_264,job_desc_265,job_desc_266,job_desc_267,job_desc_268,job_desc_269,job_desc_270,job_desc_271,job_desc_272,job_desc_273,job_desc_274,job_desc_275,job_desc_276,job_desc_277,job_desc_278,job_desc_279,job_desc_280,job_desc_281,job_desc_282,job_desc_283,job_desc_284,job_desc_285,job_desc_286,job_desc_287,job_desc_288,job_desc_289,job_desc_290,job_desc_291,job_desc_292,job_desc_293,job_desc_29

In [6]:
train_dir = train[['job_title', 'job_posted_date', 'salary_category', 'job_state', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12' ]]

### Analysis of Normal Features

In [7]:
pd.crosstab(train_dir['job_title'], train_dir['salary_category'])

salary_category,High,Low,Medium
job_title,,,
Job_Title_1,0,4,5
Job_Title_10,0,3,5
Job_Title_11,1,0,0
Job_Title_12,1,0,0
Job_Title_13,2,0,0
Job_Title_14,0,0,1
Job_Title_15,1,0,0
Job_Title_2,1,0,4
Job_Title_3,3,2,4


In [9]:
contingency_table = pd.crosstab(train_dir['job_title'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between job_title and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between job_title and salary_category.")

Chi-square statistic: 50.41408721579297
P-value: 0.011227833521336049
Degrees of freedom: 30
Reject the null hypothesis: There is a significant association between job_title and salary_category.


In [11]:
train['job_title'].value_counts()

,count
job_title,
Others,143
Job_Title_1,9
Job_Title_3,9
Job_Title_10,8
Job_Title_5,5
Job_Title_7,5
Job_Title_2,5
Job_Title_8,4
Job_Title_6,2


In [12]:
other_train = train_dir[train_dir['job_title'] == 'Others']

In [13]:
pd.crosstab(other_train['job_state'], other_train['salary_category'])

salary_category,High,Low,Medium
job_state,,,
AZ,0,1,0
CA,22,34,6
CO,1,0,1
CT,1,0,0
DC,0,0,1
FL,0,0,2
GA,1,0,0
IL,1,1,0
IN,0,1,0


In [19]:
train_dir['job_state'] = train_dir['job_state'].astype(str).replace('nan', '')

jst = []

for _, i in train_dir.iterrows():
  if i['job_title'] == 'Others':
    jst.append(i['job_state'] + i['job_title'])

  else:
    jst.append(i['job_title'])


train_dir['job_title_state'] = jst

In [20]:
pd.crosstab(train_dir['job_title_state'], train_dir['salary_category'])

salary_category,High,Low,Medium
job_title_state,,,
AZOthers,0,1,0
CAOthers,22,34,6
COOthers,1,0,1
CTOthers,1,0,0
DCOthers,0,0,1
FLOthers,0,0,2
GAOthers,1,0,0
ILOthers,1,1,0
INOthers,0,1,0


In [21]:
pd.crosstab(train_dir['job_state'], train_dir['salary_category'])

salary_category,High,Low,Medium
job_state,,,
,5,0,3
AZ,0,1,0
CA,28,40,11
CO,1,0,1
CT,1,0,0
DC,1,0,1
FL,0,0,2
GA,1,0,0
IL,2,1,0


In [22]:
train_dir['job_posted_date'] = pd.to_datetime(train_dir['job_posted_date'], format='%Y/%m')

# Extract year and month
train_dir['year'] = train_dir['job_posted_date'].dt.year
train_dir['month'] = train_dir['job_posted_date'].dt.month

In [36]:
pd.crosstab(train_dir['month'], train_dir['salary_category'])

salary_category,High,Low,Medium
month,,,
1.0,2,0,1
2.0,3,1,3
3.0,2,0,2
4.0,5,0,6
5.0,8,1,4
6.0,17,3,6
7.0,37,49,14
8.0,4,10,8
9.0,1,1,2


In [32]:
masked_df = train_dir[train_dir['year'] == 2024]

In [35]:
pd.crosstab(masked_df['job_title_state'], masked_df['salary_category'])

salary_category,High,Low,Medium
job_title_state,,,
AZOthers,0,1,0
CAOthers,18,34,4
FLOthers,0,0,2
GAOthers,1,0,0
ILOthers,1,1,0
INOthers,0,1,0
Job_Title_1,0,4,5
Job_Title_10,0,1,5
Job_Title_11,1,0,0


In [37]:
train_dir.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        200 non-null    object        
 1   job_posted_date  199 non-null    datetime64[ns]
 2   salary_category  200 non-null    object        
 3   job_state        200 non-null    object        
 4   feature_1        200 non-null    object        
 5   feature_2        200 non-null    float64       
 6   feature_3        200 non-null    bool          
 7   feature_4        200 non-null    bool          
 8   feature_5        200 non-null    bool          
 9   feature_6        200 non-null    bool          
 10  feature_7        200 non-null    bool          
 11  feature_8        200 non-null    bool          
 12  feature_9        200 non-null    bool          
 13  feature_10       134 non-null    float64       
 14  feature_11       200 non-null    bool     

In [38]:
pd.crosstab(train_dir['feature_1'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_1,,,
A,84,60,49
B,0,7,0


In [39]:
pd.crosstab(train_dir['feature_3'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_3,,,
False,69,64,41
True,15,3,8


In [47]:
contingency_table = pd.crosstab(train_dir['feature_3'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between feature 3 and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between job_title and salary_category.")

Chi-square statistic: 6.5342372746240125
P-value: 0.03811609529713968
Degrees of freedom: 2
Reject the null hypothesis: There is a significant association between feature 3 and salary_category.


In [45]:
train_dir.groupby('salary_category')['feature_2'].mean()

,feature_2
salary_category,
High,0.631129
Low,0.509966
Medium,0.619661


In [48]:
pd.crosstab(train_dir['feature_4'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_4,,,
False,82,67,45
True,2,0,4


In [49]:
pd.crosstab(train_dir['feature_5'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_5,,,
False,70,64,41
True,14,3,8


In [50]:
contingency_table = pd.crosstab(train_dir['feature_5'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between feature 5 and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between feature 5 and salary_category.")

Chi-square statistic: 5.931740713342907
P-value: 0.05151561277012124
Degrees of freedom: 2
Fail to reject the null hypothesis: There is no significant association between feature 5 and salary_category.


In [51]:
pd.crosstab(train_dir['feature_6'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_6,,,
False,28,24,14
True,56,43,35


In [52]:
contingency_table = pd.crosstab(train_dir['feature_6'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between feature 6 and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between feature 6 and salary_category.")

Chi-square statistic: 0.6800011700880498
P-value: 0.7117699063457572
Degrees of freedom: 2
Fail to reject the null hypothesis: There is no significant association between feature 6 and salary_category.


In [53]:
pd.crosstab(train_dir['feature_7'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_7,,,
False,50,38,25
True,34,29,24


In [54]:
pd.crosstab(train_dir['feature_8'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_8,,,
False,76,43,39
True,8,24,10


In [55]:
contingency_table = pd.crosstab(train_dir['feature_8'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between feature 8 and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between feature 8 and salary_category.")

Chi-square statistic: 15.549945833686007
P-value: 0.0004201188473463576
Degrees of freedom: 2
Reject the null hypothesis: There is a significant association between feature 8 and salary_category.


In [57]:
pd.crosstab(train_dir['feature_9'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_9,,,
False,84,66,48
True,0,1,1


In [58]:
train_dir.groupby('salary_category')['feature_10'].mean()

,feature_10
salary_category,
High,61.292308
Low,55.650000
Medium,46.344828


In [59]:
pd.crosstab(train_dir['feature_11'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_11,,,
False,2,5,3
True,82,62,46


In [60]:
pd.crosstab(train_dir['feature_12'], train_dir['salary_category'])

salary_category,High,Low,Medium
feature_12,,,
False,62,44,37
True,22,23,12


In [61]:
contingency_table = pd.crosstab(train_dir['feature_12'], train_dir['salary_category'])

# Calculate the Chi-square test of independence
chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
# print("Expected frequencies:\n", expected) # Uncomment to see expected frequencies

# Interpret the results
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis: There is a significant association between feature 12 and salary_category.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between feature 12 and salary_category.")

Chi-square statistic: 1.7234825673833651
P-value: 0.42242587824096545
Degrees of freedom: 2
Fail to reject the null hypothesis: There is no significant association between feature 12 and salary_category.


In [65]:
selected_columns = ['job_title', 'month', 'year', 'job_title_state','salary_category', 'job_state', 'feature_8', 'feature_2', 'feature_3', 'feature_10']

In [68]:
train_model = train_dir[selected_columns]

In [69]:
train_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   job_title        200 non-null    object 
 1   month            199 non-null    float64
 2   year             199 non-null    float64
 3   job_title_state  200 non-null    object 
 4   salary_category  200 non-null    object 
 5   job_state        200 non-null    object 
 6   feature_8        200 non-null    bool   
 7   feature_2        200 non-null    float64
 8   feature_3        200 non-null    bool   
 9   feature_10       134 non-null    float64
dtypes: bool(2), float64(4), object(4)
memory usage: 13.0+ KB


In [73]:
X = train_model.drop('salary_category', axis=1)
y = train_model['salary_category']
y = y.replace({'Low': 0, 'Medium': 1, 'High': 2})

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [74]:
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

In [114]:
# Identify numerical and categorical features
numeric_features = x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = x_train.select_dtypes(include=['object']).columns

# Create separate pipelines for numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine the pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    # Set remainder='passthrough' to include columns not explicitly listed
    # In this case, we should ensure all relevant columns are handled by num or cat transformers
    # If there were other columns to pass through, this would be useful.
    # Here, all columns are either numeric or categorical.
    remainder='passthrough' # Can be 'drop' if no other columns
)

# Final pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC())
])

# Fit model
model.fit(x_train, y_train)

# Predict
y_pred = model.predict(x_test)

print(accuracy_score(y_test, y_pred))

0.7


In [116]:
# prompt: Let us see how good the model is, let us train it on 5 folds and see the cv

from sklearn.model_selection import cross_val_score, KFold

# Define the number of folds for cross-validation
n_splits = 5

# Create a KFold object
kf = KFold(n_splits=n_splits, shuffle=True, random_state=420)

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

print(f"Cross-validation scores (Accuracy) for {n_splits} folds:")
for fold, score in enumerate(cv_scores):
    print(f"Fold {fold+1}: {score:.4f}")

print(f"\nMean CV Accuracy: {cv_scores.mean():.4f}")
print(f"Standard Deviation of CV Accuracy: {cv_scores.std():.4f}")

Cross-validation scores (Accuracy) for 5 folds:
Fold 1: 0.6500
Fold 2: 0.6000
Fold 3: 0.7250
Fold 4: 0.6750
Fold 5: 0.5500

Mean CV Accuracy: 0.6400
Standard Deviation of CV Accuracy: 0.0604
